In [ ]:
import json
import sqlite3
import pandas as pd
import requests
import matplotlib.pyplot as plt


In [ ]:
#DB_FILE = "../data/xss_research.db"
DB_FILE = "../data/xss_research_v2.db"
conn = sqlite3.connect(DB_FILE)

In [ ]:
FRAMEWORKS_OF_INTEREST = ['clientside_javascript', 'django', 'express', 'flask', 'go', 'java_and_jsp', 'javascript', 'rails', 'react']

In [ ]:
c = conn.cursor()
it = c.execute("SELECT repo_url, fix_commit, previous_commit, triage_status, reviewer_notes, taxonomy FROM finding")
triage_data_df = pd.DataFrame(list(it), columns=["repo_url", "fix_commit", "previous_commit", "triage_status", "reviewer_notes", "taxonomy"])

# rename 'false_positive' to 'false_negative' due to naming error :3
triage_data_df['triage_status'] = triage_data_df['triage_status'].replace('false_positive','false_negative')

In [ ]:
import csv
with open("../data/framework-labels/xss-labeled-frameworks-1611967876.46879.csv", 'r') as fin:
    framework_df = pd.read_csv(fin)
with open("../data/framework-labels/new-xss-frameworks-done.csv", 'r') as fin:
    framework_df2 = pd.read_csv(fin)

framework_df = framework_df.append(framework_df2)

In [ ]:
framework_df = framework_df[['repo_url', 'framework']]
framework_df = framework_df.drop_duplicates('repo_url')

In [ ]:
m = framework_df.merge(triage_data_df, on="repo_url").drop_duplicates(['fix_commit'])

In [ ]:
# Update rows based on round 2 manual analysis
# Only do this if round 2
if "v2" in DB_FILE:
    with open("../data/framework-labels/xss-framework-label-round2-updates.csv", 'r') as fin:
        update_df = pd.read_csv(fin)
    for row in update_df.itertuples(index=False):
        m.loc[(m.repo_url == row[0]) & (m.fix_commit == row[1]), 'framework'] = row[2]
        m.loc[(m.repo_url == row[0]) & (m.fix_commit == row[1]), 'triage_status'] = row[3]

In [ ]:
# drop anything that is 'unknown' or slated for deletion
m = m[(m.triage_status == "true_positive") | (m.triage_status == "false_negative")]

In [ ]:
# filter to frameworks of interest
m = m.loc[m['framework'].isin(FRAMEWORKS_OF_INTEREST)]
g = m.groupby('triage_status').size()
g

In [ ]:
print("commits: ", len(set(m.fix_commit)))

In [ ]:
print("repos: ", len(set(m.repo_url)))

In [ ]:
print("detection rate: ", g['true_positive'] / len(set(m.fix_commit)))

In [ ]:
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['figure.dpi'] = 100
p = m[['framework', 'triage_status', 'fix_commit', 'taxonomy']].groupby(['framework', 'triage_status']).agg({'fix_commit': 'count'})
p

In [ ]:
p.unstack('triage_status')

In [ ]:
x = p.unstack('triage_status').reindex(['go', 'rails', 'react', 'javascript', 'django', 'java_and_jsp', 'flask', 'express', 'clientside_javascript']).plot(kind='bar', stacked=True, colormap="RdYlGn", rot=75)
matplotlib.pyplot.legend(["False Negative", "True Positive"])
x

In [ ]:
q = m.groupby(['framework']).agg({'fix_commit': 'count'})

In [ ]:
r = p.div(q, level='framework')
z = r.reset_index().merge(q, on=['framework']).set_index(['framework', 'triage_status'])

In [ ]:
y = z[(z.index.get_level_values(1) == "true_positive") & (z.index.get_level_values(0).isin(FRAMEWORKS_OF_INTEREST))].sort_values(by="fix_commit_x", ascending=False)
y.columns = ['%', 'N']
y